# 导入模块

In [1]:
import numpy as np
import pandas as pd
import sys
import re
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 函数定义

In [2]:
# 导入数据函数
path = r'L:\项目配仓\兰州石化职业技术大学\二期\04模型及实施过程\招培就收集数据模板\\'
def get_ori_data(name):
    data_ori = pd.read_excel(path+name)
#     print(data_ori.head(1))
    return data_ori

In [3]:
# 熵值法
def cal_weight(column,row,value):
    p= np.array([[0.0 for i in range(column)] for i in range(row)])     
    for i in range(column):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
    e=-1/np.log(row)*sum(p*np.log(p+1e-10))      #计算熵值
    g=1-e     # 计算一致性程度
    w=g/sum(g)     #计算权重
    return w

In [4]:
# 归一化
def change_1(data_need):
    min_max_model = MinMaxScaler()
    data_f = min_max_model.fit_transform(data_need)
    d_columns = data_need.columns
    d_index = data_need.index
    return pd.DataFrame(list(data_f), columns=d_columns, index=d_index)

In [5]:
# 算分并合并
def merge_data(excel_name,score_name,f_name):
    data_ori = get_ori_data(excel_name)
    data_ori[score_name] = data_ori.apply(f_name,axis=1)
    data_ori =data_ori[['学号',score_name]].groupby('学号').sum().reset_index()
    global data_sum
    data_sum = pd.merge(data_sum, data_ori, how='outer', on='学号')#.fillna(0)
    
#     print('---------------------')
#     print(data_sum.head(1))
#     print('---------------------')

# 合并数据

In [6]:
# 到时候数据不干净先洗一下，不要改dict
dict ={'处分种类':{'轻微':1,'严重':2},
       '项目等级':{'校级':1,'省级':2},
       '部门名称':{'纪律部':1,'xx部':2},
       '职位名称':{'干事':1,'xx':2},
       '任职时长':{'一年':1,'两年':2},
       '比赛等级':{'校级':1,'国家级':2},
       '奖项':{'一等奖':3,'二等奖':2,'三等奖':1}
      }

In [7]:
data_sum = pd.read_excel('学生名单.xlsx')
merge_data('惩罚记录.xlsx','惩罚',lambda x:dict['处分种类'][x['处分种类']])

merge_data('科研记录.xlsx','科研',lambda x: 2*dict['项目等级'][x['项目等级']] if x['负责人学号']== x['学号'] else dict['项目等级'][x['项目等级']])

merge_data('学生干部.xlsx','干部',lambda x:dict['部门名称'][x['部门名称']]*dict['职位名称'][x['职位名称']]*dict['任职时长'][x['任职时长']])

merge_data('荣誉档案.xlsx','荣誉',lambda x:dict['比赛等级'][x['比赛等级']]*dict['奖项'][x['奖项']])

merge_data('实习实践.xlsx','实习',lambda x:int(re.findall("\d+",x['实践时长'])[0]))

merge_data('社会实践.xlsx','社会',lambda x:int(re.findall("\d+",x['参与时长（分）'])[0]))

In [8]:
data_sum = data_sum.fillna(0)
temp_list = ['社会','实习','荣誉','干部','科研','惩罚']
data_sum[temp_list] = change_1(data_sum[temp_list])
data_sum['惩罚'] = 1 - data_sum['惩罚']

In [9]:
w = cal_weight(5,data_sum.shape[0],data_sum[temp_list].values)

In [10]:
data_sum['培养总分'] = 0
for i in range(5):
    data_sum['培养总分'] = data_sum['培养总分'] + w[i]*data_sum[temp_list[i]]
data_sum

,专业,学号,惩罚,科研,干部,荣誉,实习,社会,培养总分
0,计算机,（必填）2022080301,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,数媒,（必填）202208031,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# 分组数据

In [11]:
data_group =data_sum[['专业','社会','实习','荣誉','干部','科研','惩罚']].groupby('专业').mean().reset_index()

In [12]:
data_group['培养总分'] = 0
for i in range(5):
    data_group['培养总分'] = data_group['培养总分'] + w[i]*data_group[temp_list[i]]
data_group

,专业,社会,实习,荣誉,干部,科研,惩罚,培养总分
0,数媒,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,计算机,1.0,1.0,1.0,1.0,1.0,0.0,1.0
